In [1]:
from life_after_bert.data import load_olmpics_data, MCDataset
from life_after_bert.eval import evaluate_encoder_decoder

In [2]:
import numpy as np
import torch
import transformers

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
model_name = "t5-large"
task = "oLMpics MLM"
data_path = "../tests/data/oLMpics_age_comparison_dev.jsonl"
num_choices = 2
mask_token = "<extra_id_0>"

In [4]:
model = transformers.T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()
model.to(device)
pass  # Get rid of print

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, mask_token=mask_token)  # Add mask_token to vocabulary
if tokenizer.pad_token == None:
    tokenizer.pad_token = tokenizer.eos_token

questions, choice_lists, answer_ids = load_olmpics_data(data_path, num_choices)
dataset = MCDataset(questions, choice_lists, answer_ids, tokenizer, mask_token)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 572992.35it/s]


In [6]:
decoder_prompt = tokenizer("<pad> <extra_id_0>", add_special_tokens=False, return_tensors="pt").input_ids

In [7]:
all_answers, all_preds = evaluate_encoder_decoder(model, tokenizer, task, dataset, decoder_prompt, device)
(np.array(all_answers) == np.array(all_preds)).mean()

Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:02<00:00, 15.25it/s]


0.94